In [ ]:
from pymongo import MongoClient

# Replace the following with your MongoDB connection details
MONGO_URI = "mongodb://127.0.0.1:27017/?directConnection=true&serverSelectionTimeoutMS=2000&appName=mongosh+2.3.3"

# Connect to MongoDB
client = MongoClient(MONGO_URI)

# Access the database
db_name = "tutor"  # Replace with your database name
db = client[db_name]

# Access the collection
collection_name = "embeddings"  # Replace with your collection name
collection = db[collection_name]

# Example: Fetch all documents from the collection
documents = collection.find()

# Print the documents
for doc in documents:
    print(doc)

# Close the connection
client.close()


In [7]:
!pip install --upgrade openai


  Attempting uninstall: openai
    Found existing installation: openai 1.53.0
    Uninstalling openai-1.53.0:
      Successfully uninstalled openai-1.53.0


In [ ]:
import pandas as pd
from openai import OpenAI
client = OpenAI(api_key="sk-hfXMFFi2WEXRerJJalmD7EkUhMK_D3UQmhcKJuo-2-T3BlbkFJetMKDlN69wvd4XYjPqSkKgqKXyPI_5_REXMgJwO7YA")
from pymongo import MongoClient


# Step 2: Load the CSV file
csv_file_path = "gemini_complete.csv"  # Replace with the path to your CSV file
df = pd.read_csv(csv_file_path,encoding='ISO-8859-1')

# Ensure the 'question' column exists
if 'Question' not in df.columns:
    raise ValueError("The CSV file does not contain a 'question' column.")

# Step 3: Extract the 'question' column
questions = df['Question'].tolist()

# Step 4: Generate embeddings using OpenAI
def get_embedding(text, model="text-embedding-ada-002"):
    response = client.embeddings.create(input=text, model=model)
    return response.data[0].embedding

# Generate embeddings for all questions
embeddings = [get_embedding(question) for question in questions]

# Step 5: Store embeddings in MongoDB
# MongoDB connection details
MONGO_URI = "mongodb://localhost:27017"  # Replace with your MongoDB connection string
client = MongoClient(MONGO_URI)

# Access the database and collection
db = client["tutor"]  # Replace with your database name
collection = db["openai_embedding"]  # Replace with your collection name

# Prepare documents for MongoDB
documents = [
    {"question": question, "embedding": embedding}
    for question, embedding in zip(questions, embeddings)
]

# Insert documents into the MongoDB collection
collection.insert_many(documents)

print(f"Inserted {len(documents)} documents into MongoDB!")

# Close the MongoDB connection
client.close()


TypeError: 'CreateEmbeddingResponse' object is not subscriptable